In [149]:
import pandas as pd
import numpy as np
import catboost
from sklearn.model_selection import train_test_split
import seaborn as sns

In [193]:
df = pd.read_csv('gender.csv')

In [194]:
df = df[df.columns[2:12]]

In [195]:
df = df.drop(['forehead_width_mm', 'forehead_width_conventional_units'], axis=1)

In [196]:
df.columns

Index(['long_hair', 'forehead_width_cm', 'forehead_height_cm', 'nose_wide',
       'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'gender'],
      dtype='object')

In [197]:
pd.set_option('display.max_columns', None)

In [198]:
df['gender'] = df['gender'].map({'Male':1, 'Female':0})

In [199]:
list_to_bool = ['nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'long_hair' ]

In [200]:
df

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,0.704275,11.8,6.1,0.895614,0.477485,0.841261,0.902628,NaN
1,0.118690,14.0,5.4,0.202520,0.186825,0.719697,0.309122,NaN
2,0.203894,11.8,6.3,0.892793,0.723152,0.725821,0.720640,NaN
3,0.386228,14.4,6.1,0.024963,0.562005,0.831515,0.770379,NaN
4,0.564020,13.5,5.9,0.415389,0.063705,0.211584,0.157217,NaN
...,...,...,...,...,...,...,...,...
2510,0.915744,13.9,6.4,0.604791,0.851645,0.766959,0.763820,1.0
2511,0.820041,13.8,5.1,0.933549,0.537350,0.918460,0.938982,1.0
2512,0.659143,12.2,5.3,0.531385,0.456792,0.489438,0.457138,0.0
2513,0.664928,12.1,5.2,0.538634,0.610885,0.977585,0.942940,1.0


In [201]:
for i in list_to_bool:
    df[i] = df[i].apply(lambda x:1 if 0.5 <= x < 1 else 0  )

In [202]:
test = df[df.gender.isna()]
train = df[~df.gender.isna()]


In [205]:
train

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
1001,1,13.4,6.1,1,1,1,1,1.0
1002,0,13.5,6.4,1,1,1,1,1.0
1003,1,13.4,5.4,1,0,0,0,0.0
1004,1,14.1,6.3,0,0,0,0,0.0
1005,1,12.5,5.2,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...
2510,1,13.9,6.4,1,1,1,1,1.0
2511,1,13.8,5.1,1,1,1,1,1.0
2512,1,12.2,5.3,1,0,0,0,0.0
2513,1,12.1,5.2,1,1,1,1,1.0


In [227]:
X = train[train.columns[:-1]].values
y = train['gender'].values
X_test = test[test.columns[:-1]]

In [228]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [229]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [261]:
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
model.predict(X_val)
confusion_matrix(y_val, model.predict(X_val))

array([[140,   5],
       [  7, 151]], dtype=int64)

In [262]:
cat_model = catboost.CatBoostClassifier(silent=True, random_state=42 )
cat_model.fit(X_train, y_train)


In [263]:
confusion_matrix(y_val, cat_model.predict(X_val))

array([[143,   2],
       [  7, 151]], dtype=int64)

In [264]:
cat_model.predict(X_val)

array([0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 0.

In [269]:
fin_model = catboost.CatBoostClassifier(silent=True, random_state=42 )
fin_model.fit(X, y)
predict = fin_model.predict(X_test)

In [270]:
sub = pd.DataFrame(predict)

In [271]:
sub[0] = sub[0].map({1.0:'Male', 0.0:'Female'})
sub.to_csv('fin.csv', index=False )

In [272]:
pd.read_csv('fin.csv')

,0
0,Male
1,Female
2,Male
3,Male
4,Female
...,...
996,Female
997,Female
998,Male
999,Male
